In [1]:
import sqlite3
import pandas as pd
import numpy as np

Define all paths

In [2]:
general_path = 'fmnist-exp/Baseline_exps/'
experiment_paths = ['Base1/', 'Base2/', 'Base3/', 'Base4/', 'Base5/']
epoch_paths = ['First10/', 'Second10/', 'Third10/', 'Fourth10/', 'Fifth10/', 'Sixth10/', 'Seventh10/']

Connect to all client databases, read the statistics table and create a DataFrame.

In [3]:
def _get_statistics_table(connection_path, client_flag):
    if client_flag:
        conn = sqlite3.connect(connection_path)
        df = pd.read_sql_query(sql='SELECT * FROM statistics', con=conn)
        conn.close()
        return df
    else:
        conn = sqlite3.connect(connection_path)
        df = pd.read_sql_query(sql='SELECT * FROM statistics', con=conn)
        df1, df2 = df[df['client_id'] == 1], df[df['client_id'] == 2]
        conn.close()
        return [df1, df2]

Do some minor data trasnformation to get the desired format and merge the DataFrames.

In [4]:
def _merge_dfs(df_list, client_flag):
    df = pd.concat(df_list, axis=0)
    df['epoch'] = np.arange(0, 70)
    df.set_index(keys='epoch', inplace=True)
    (df.drop(['testing_acc', 'gamma'], axis=1, inplace=True)) if client_flag else df.drop(columns='server_test_acc', axis=1, inplace=True)     
    return df

Iterate over each experiment, extract and export the data in .csv format.

In [5]:
def _retrive_exp_data(data_path):
    c0_dfs = []
    c1_dfs = []
    c0_server_dfs = []
    c1_server_dfs = []
    for path in epoch_paths:
        c0_dfs.append(_get_statistics_table(data_path + path + 'database_0.db', True))
        c1_dfs.append(_get_statistics_table(data_path + path + 'database_1.db', True))
        server_dfs = (_get_statistics_table(data_path + path + 'server_1_database.db', False))
        c0_server_dfs.append(server_dfs[0])
        c1_server_dfs.append(server_dfs[1])
    c0_df = _merge_dfs(c0_dfs, True)
    c1_df = _merge_dfs(c1_dfs, True)
    c0_server_df = _merge_dfs(c0_server_dfs, False)
    c1_server_df = _merge_dfs(c1_server_dfs, False)
    final_c0_df = c0_df.join(other=c0_server_df, lsuffix='_c0_df', rsuffix='_c0_server_df')
    final_c1_df = c1_df.join(other=c1_server_df, lsuffix='_c1_df', rsuffix='_c1_server_df')
    final_c0_df.to_csv(data_path + 'client0.csv')
    final_c1_df.to_csv(data_path + 'client1.csv')

In [6]:
for idx, path in enumerate(experiment_paths):
    _retrive_exp_data(general_path + path)
    print(f'[+] Exported {idx}th experiment data successfully.')
    
    

[+] Exported 0th experiment data successfully.
[+] Exported 1th experiment data successfully.
[+] Exported 2th experiment data successfully.
[+] Exported 3th experiment data successfully.
[+] Exported 4th experiment data successfully.
